In [ ]:

import pandas as pd
from torch.utils.data import Dataset

from torch.utils.data import DataLoader
import torch
from torchvision import models
import torch
import pandas as pd
from PIL import Image
import torch.nn as nn
import torch
import torchvision.transforms as transforms
from PIL import Image
from transformers import AutoTokenizer, AutoModel
import pandas as pd

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [ ]:
"""
テキスト処理のモデル
"""
class CaptionEncoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese-v2")
  def forward(self, x):
    x = self.bert(x)
    x = torch.max(x.last_hidden_state, dim=1)[0]  # max pooling
    return x

In [ ]:
model_name =  'caption_image_2023-06-28.pth'
caption_model = CaptionEncoder().to(device)
caption_model.load_state_dict(torch.load(model_name))
caption_model.eval()

In [ ]:
class FDataset(Dataset):
    def __init__(self, annotations_file):
        self.data = pd.read_csv(annotations_file) 
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        caption = self.data.iloc[idx, 1]
        return caption

In [ ]:
dataset = FDataset('./data/anotation_new.csv')
dataloader =  DataLoader(dataset, batch_size=256, shuffle=False, pin_memory=True, num_workers=0)
caption_embeddings = torch.Tensor([])


In [ ]:
for i, (cap) in enumerate(dataloader):
    # 予測と損失の計算
    cap = cap.to(device)
    with torch.no_grad():
        caption_embedding = caption_model(cap).cpu()
    caption_embeddings = torch.cat((caption_embeddings, caption_embedding), dim=0)
    if i % 100 == 0:
        print(f"finish: {len(caption_embeddings) * 100/len(dataset)}%")

torch.save(caption_embeddings, f'caption_tensor/{model_name[:-3]}/resnet50_tensor.pt')